# Neural Networks from scratch


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

## Build Neural Network


In [ ]:
class BaseLayer:
    def __init__(self):
        self.input = None
        self.output = None

    def forward(self, input):
        """
        Abstract method for forward method.
        """
        pass

    def backward(self, output_gradient, learning_rate):
        """
        Abstract method for backward method.
        """
        pass

In [ ]:
"""
Assume that we have a neural network layer with i inputs and j outputs
- The weights matrix will be of shape (j x i)
- The input vector will be of shape (i x 1)
- The bias vector will be of shape (j x 1)
Finally, the output vector will be calculate by dot product and vector addition:
> weights . input + bias = (j x i) . (i x 1) + (j x 1) = (j x 1)
"""


class DenseLayer(BaseLayer):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.weights = np.random.randn(output_size, input_size)  # j x i
        self.bias = np.random.randn(output_size, 1)  # j x 1

    def forward(self, input):
        self.input = input
        output = np.dot(self.weights, self.input) + self.bias
        return output

    def backward(self, output_gradient, learning_rate):
        weights_gradient = np.dot(output_gradient, self.input.T)

        # Derivative of loss respect to input
        derivative = np.dot(self.weights.T, output_gradient)

        # Apply gradient descent
        self.weights -= learning_rate * weights_gradient
        self.bias -= learning_rate * output_gradient

        return derivative

In [ ]:
class ActivationLayer(BaseLayer):
    def __init__(self, activation, activation_derivative):
        super().__init__()
        self.activation = activation  # Activation function (e.g., sigmoid, relu)
        self.activation_derivative = (
            activation_derivative  # Derivative of activation function
        )

    def forward(self, input):
        self.input = input
        activated_output = self.activation(input)
        return activated_output

    def backward(self, output_gradient, learning_rate=None):
        """
        Apply chain rule: multiply output gradient by activation derivative

        Note: learning_rate is not used since activation layers have no parameters to update
        """
        derivative = self.activation_derivative(self.input)
        return np.multiply(output_gradient, derivative)

In [ ]:
class Tanh(ActivationLayer):
    def __init__(self):
        def tanh(x):
            return np.tanh(x)

        def tanh_derivative(x):
            return 1 - np.tanh(x) ** 2

        super().__init__(tanh, tanh_derivative)

In [ ]:
def mean_squared_error(y_actual, y_pred):
    """
    Mean Squared Error (MSE) loss function
    """
    return np.mean((y_actual - y_pred) ** 2)


def mean_squared_error_derivative(y_actual, y_pred):
    """
    Derivative of Mean Squared Error (MSE) loss function
    """
    return 2 * (y_pred - y_actual) / np.size(y_actual)

## Data Exploration


In [ ]:
df = pd.read_csv("data/ObesityDataSet_raw_and_data_sinthetic.csv")
df.head()

In [ ]:
# Check duplicates
df.duplicated().sum()

In [ ]:
# Check missing values
df.isnull().sum()

In [ ]:
df.describe()

## Data Preprocessing


In [ ]:
# Remove duplicates
df = df.drop_duplicates()

In [ ]:
df.duplicated().sum()

## Plot the initial data


In [ ]:
# Plot distribution of obesity classes
plt.figure(figsize=(12, 6))
obesity_counts = df["NObeyesdad"].value_counts().sort_index()
sns.barplot(x=obesity_counts.index, y=obesity_counts.values)
plt.title("Distribution of Obesity Classes")
plt.xticks(rotation=45)
plt.ylabel("Count")
plt.tight_layout()
plt.show()

In [ ]:
# Weight vs. Height colored by obesity class
plt.figure(figsize=(10, 8))
for label, group in df.groupby("NObeyesdad"):
    plt.scatter(group["Height"], group["Weight"], label=label, alpha=0.7)
plt.title("Weight vs. Height by Obesity Class")
plt.xlabel("Height (m)")
plt.ylabel("Weight (kg)")
plt.legend(title="Obesity Class", bbox_to_anchor=(1, 1))
plt.tight_layout()
plt.grid(alpha=0.3)
plt.show()

In [ ]:
# Age distribution by gender and obesity class
plt.figure(figsize=(14, 8))
for i, gender in enumerate(["Male", "Female"]):
    plt.subplot(1, 2, i + 1)
    data = df[df["Gender"] == gender]
    sns.boxplot(
        x="NObeyesdad", y="Age", data=data, order=sorted(df["NObeyesdad"].unique())
    )
    plt.title(f"Age Distribution for {gender}")
    plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Transportation method by obesity class
plt.figure(figsize=(14, 8))
pd.crosstab(df["MTRANS"], df["NObeyesdad"]).plot(kind="bar", stacked=True)
plt.title("Transportation Method vs. Obesity Class")
plt.xlabel("Transportation Method")
plt.xticks(rotation=45)
plt.ylabel("Count")
plt.legend(title="Obesity Class", bbox_to_anchor=(1, 1))
plt.tight_layout()
plt.show()

In [ ]:
# Water consumption by obesity class
plt.figure(figsize=(14, 8))
sns.boxplot(x="NObeyesdad", y="CH2O", data=df, order=sorted(df["NObeyesdad"].unique()))
plt.title("Water Consumption by Obesity Class")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Family history of overweight by obesity class
plt.figure(figsize=(12, 6))
family_obesity = pd.crosstab(
    df["family_history_with_overweight"], df["NObeyesdad"], normalize="index"
)
family_obesity.plot(kind="bar", stacked=False)
plt.title("Family History of Overweight vs. Obesity Class")
plt.xlabel("Family History with Overweight")
plt.ylabel("Percentage")
plt.xticks(rotation=0)
plt.legend(title="Obesity Class")
plt.tight_layout()
plt.show()